In [2]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Install Required Libraries
!pip install tensorflow opencv-python

# Step 3: Import Libraries
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

# Step 4: Define Constants
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 20
DATASET_PATH = "/content/drive/MyDrive/ColabNotebooks/UTKFace/"

# Step 5: Verify Dataset Path
if not os.path.exists(DATASET_PATH):
    raise FileNotFoundError(f"Dataset folder not found at {DATASET_PATH}. Please check the path.")

# Step 6: Function to Group Ages into Bins
def age_to_group(age):
    if age <= 10:
        return 0
    elif 11 <= age <= 20:
        return 1
    elif 21 <= age <= 30:
        return 2
    elif 31 <= age <= 40:
        return 3
    elif 41 <= age <= 50:
        return 4
    elif 51 <= age <= 60:
        return 5
    else:
        return 6

# Step 7: Get All Image Paths and Labels
image_paths = [os.path.join(DATASET_PATH, img) for img in os.listdir(DATASET_PATH) if img.endswith(".jpg")]
labels = [age_to_group(int(img.split("_")[0])) for img in os.listdir(DATASET_PATH) if img.endswith(".jpg")]

# Step 8: Convert Labels to One-Hot Encoding
label_binarizer = LabelBinarizer()
labels = label_binarizer.fit_transform(labels)

# Step 9: Split into Training & Validation Sets
train_paths, val_paths, train_labels, val_labels = train_test_split(image_paths, labels, test_size=0.15, random_state=42)

# Step 10: Custom Data Generator
class UTKFaceDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, image_paths, labels, batch_size=BATCH_SIZE, img_size=(IMG_SIZE, IMG_SIZE), augment=False):
        self.image_paths = image_paths
        self.labels = labels
        self.batch_size = batch_size
        self.img_size = img_size
        self.augment = augment

    def __len__(self):
        return int(np.floor(len(self.image_paths) / self.batch_size))

    def __getitem__(self, index):
        batch_paths = self.image_paths[index * self.batch_size:(index + 1) * self.batch_size]
        batch_labels = self.labels[index * self.batch_size:(index + 1) * self.batch_size]

        images = []
        for img_path in batch_paths:
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, self.img_size)
            img = img / 255.0

            if self.augment:
                if np.random.rand() > 0.5:
                    img = cv2.flip(img, 1)
                img = tf.keras.preprocessing.image.random_rotation(img, 30)
                img = tf.keras.preprocessing.image.random_shift(img, 0.2, 0.2)

            images.append(img)

        return np.array(images), np.array(batch_labels)

# Step 11: Create Data Generators
train_generator = UTKFaceDataGenerator(train_paths, train_labels, augment=True)
val_generator = UTKFaceDataGenerator(val_paths, val_labels)

# Step 12: Load Pre-Trained ResNet50 Model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Step 13: Freeze Base Model Layers
for layer in base_model.layers:
    layer.trainable = False

# Step 14: Add Custom Layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(len(label_binarizer.classes_), activation='softmax')(x)

# Step 15: Create Final Model
model = Model(inputs=base_model.input, outputs=predictions)

# Step 16: Compile Model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Step 17: Train Model
history = model.fit(train_generator, validation_data=val_generator, epochs=EPOCHS)

# Step 18: Save Model
model.save("/content/drive/MyDrive/age_prediction_model.h5")

# Step 19: Evaluate Model
test_loss, test_accuracy = model.evaluate(val_generator)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

Mounted at /content/drive
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 136s 8s/step - accuracy: 0.8150 - loss: 0.7485 - val_accuracy: 0.9062 - val_loss: 0.3385
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 118s 7s/step - accuracy: 0.7961 - loss: 0.6348 - val_accuracy: 0.9062 - val_loss: 0.3196
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 149s 8s/step - accuracy: 0.7997 - loss: 0.6408 - val_accuracy: 0.9062 - val_loss: 0.3218
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 117s 7s/step - accuracy: 0.8620 - loss: 0.5118 - val_accuracy: 0.9062 - val_loss: 0.3471
Epoch 5/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 125s 8s/step - accuracy: 0.7600 - loss: 0.6522 - val_accuracy: 0.9062 - val_loss: 0.3387
Epoch 6/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 125s 7s/step - accuracy: 0.8445 - loss: 0.5846 - val_accuracy: 0.9062 - val_loss: 0.3633
Epoch 7/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 117s 7s/step - accuracy: 0.8082 - loss: 0.6153 - val_accuracy: 0.9062 - val_loss: 0.3219
Epoch 8/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 119s 7s/step - accuracy: 0.8443 - loss: 0.5720 - val_accuracy: 0.9062 - v

2/2 ━━━━━━━━━━━━━━━━━━━━ 12s 5s/step - accuracy: 0.8958 - loss: 0.3534
Test Loss: 0.3350868225097656, Test Accuracy: 0.90625
